# Projeto 1 - Ciência dos Dados

Nome: Arthur Cisotto Machado

Nome: Alessandra Yumi Carvalho Ogawa

#### CONTEXTO DO PROJETO 

A empresa de streaming *Netflix* em parceria com o estúdio espanhol *Vancouver Media* deseja saber e analisar como a audiencia está reagindo a série de sucesso *La Casa de Papel* na rede social Twitter. 
O projeto exige a criação de um programa que consiga classificar os tweets entre **relevantes** ou **irrelevantes** para a análise da empresa.

A classificação foi feita com o intúito de ajudar a área de marketing das duas empresas parceiras a acharem comentários que possam ser úteis em algum sentido estratégio para mudança de operações internas e também como fonte de *feedback* em relação ao conteúdo cinematográfico produzido.

___
#### CARREGANDO AS BIBLIOTECAS UTILIZADAS NO RPOGRAMA:

In [43]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
import emoji
from emoji import UNICODE_EMOJI

In [44]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/alessandrayumiogawa/Documents/INSPER/INSPER - 2A/C-DADOS/P1_CDados


___
#### FUNÇÃO DE LIMPEZA DOS TWEETS:
- tira sinais de pontuação irrelevantes para o texto;
- exculi repetição de emoji;
- substitui emoji por seu código;

In [45]:
# https://docs.python.org/3/library/re.html#


def cleanup(text):

    punctuation = '[”/-@\\n:;?\"\'().,]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    text_split = text_subbed.split()
    return ' '.join(text_split).lower()

def limpa_emoji(tweet):
    
    modified=' '.join(emoji.get_emoji_regexp().split(tweet))
    modified=modified.split()
    for i,emoji1 in enumerate(modified):
        if emoji1 in UNICODE_EMOJI['pt']:
            modified[i]=UNICODE_EMOJI['pt'][emoji1].replace(':','')
        elif emoji1 in UNICODE_EMOJI['en']:
            modified[i]=UNICODE_EMOJI['en'][emoji1].replace(':','')
        else:
            continue
    modified=' '.join(modified)
        
    return modified

In [46]:
def limpeza_e_emoji(tweet):                 #compilado das duas funções em uma só para mais fácil aplicação
    texto_filtrado = cleanup(tweet)
    sem_emoji = limpa_emoji(texto_filtrado)
    return sem_emoji

___
#### CARREGANDO A BASE DE DADOS COM OS TWEETS CLASSIFICADOS COMO ***RELEVANTES*** E ***IRRELEVANTES*** MANUALMENTE:

In [47]:
filename = 'la casa de papel.xlsx'

Separação do documento em dois DataFrames diferentes: **Treinamento** e **Teste**.

- **Treinamento**: composto por 300 tweets classificados manualmente, será usado para *ensinar* o programa a classificar um tweets de acorodo com a sua relevância levando em conta as palavras em seu conteúdo.

- **Teste**: composto por 200 tweets classificados manualmente, será usado para *testar* o programa classificador e comparar o resultado com a classificação feita à mão anteriormente. 

In [48]:
train = pd.read_excel(filename)
train['Treinamento'] = train['Treinamento'].apply(limpeza_e_emoji) #aplicando a função de limpeza
train.head(5)


,Treinamento,"Classificação (relevante = 1, não relevante = 0)"
0,tô vendo la casa de papel mdssss o primeiro ep...,1.0
1,agora vou assistir lá casa de papel e dormir p...,0.0
2,dedo coçando pra assistir lá casa de papel mas...,0.0
3,la casa de papel me faz torcer pros bandidos,1.0
4,la casa de papel acabou comigo me encontro des...,1.0


**OBS**: a função de limpeza já foi aplicada no DataFrame de treinamento.

In [49]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)


,Teste,"Classificação (relevante = 1, não relevante = 0)"
0,quero terminar de ver a nova parte de la casa ...,0
1,não existe outra série no mundo que mexa comig...,1
2,tá foda de desviar de todos os spoilers de la ...,1
3,@todoroki_jun2 né tô me sentindo em lá casa de...,0
4,la casa de papel é tão ruim que nem consegui t...,1


___
#### CLASSIFICADOR AUTOMÁTICO DE SENTIMENTO:


O filtro criado para a realização da classificação manual seguia o padrão especificado abaixo:

**RELEVANTES**: As mensagens de texto com relevância mostravam a opinião e sentimentos, sejam eles positivos ou negativos, sobre a série.
*Consideramos tweets relacionados a spoilers como relevantes pois eles também demonstram um forte sentimento que as pessoas possuem em relação à série.

**IRRELEVANTES**: Classificamos como não relevantes tweets que não se encaixaram na nossa classificação de relevância, tweets que falavam sobre tópicos pessoais ou tweets que falavam sobre algum personagem específico da série.



___
#### MONTANDO UM CLASSIFICADOR NAÏVE-BAYES:


- PREPARANDO OS DADOS DO TREINAMENTO

In [50]:
#criando filtros para smeprar os tweets relevantes dos irrelevantes

filtro_nao_relevante = train['Classificação (relevante = 1, não relevante = 0)']==0
filtro_relevante = train['Classificação (relevante = 1, não relevante = 0)']==1


In [51]:
#separando em dois DFs diferentes os tweets relevantes e irrelevantes

relevantes_train = train[filtro_relevante]
nao_relevantes_train = train[filtro_nao_relevante]

In [52]:
#criando uma sting grande para armazenar todos os tweets de cada classificação

relevantes_train_txt = ''
for tweet in relevantes_train['Treinamento']:
    relevantes_train_txt += " "
    relevantes_train_txt += str(tweet)

nao_relevantes_train_txt = ''
for tweet in nao_relevantes_train['Treinamento']:
    nao_relevantes_train_txt += " "
    nao_relevantes_train_txt += str(tweet)

In [53]:
#organizando os tweets em lista de palavras e pandas Series 

#fazendo a lista com todas as palavaras dos tweets
todas_palavras_relevantes = relevantes_train_txt.split()
todas_palavras_irrelevantes = nao_relevantes_train_txt.split()

#transformando a lista em uma panda Series
serie_relevante = pd.Series(todas_palavras_relevantes)
serie_irrelevante = pd.Series(todas_palavras_irrelevantes)


- PREPARANDO AS TABELAS DE FREQUÊNCIA

In [65]:
tabela_relevante = serie_relevante.value_counts()
tabela_irrelevante = serie_irrelevante.value_counts()

In [66]:
#organizando o conjunto universo de todas as palavras

palavras = relevantes_train_txt + nao_relevantes_train_txt
todas_palavras = palavras.split()
serie_palavras = pd.Series(todas_palavras)
tabela_palavras = serie_palavras.value_counts(normalize=True)
palavras_sem_repeticao = set(todas_palavras_relevantes + todas_palavras_irrelevantes)

- MONTANDO AS PROBABILIDADES

In [68]:
#calculando a probabilidade de um tweet ser classificado como relevante ou irrelevante
probR = len(serie_relevante)/len(serie_palavras)
probIR = len(serie_irrelevante)/len(serie_palavras)

In [57]:
#PRIMEIRA FUNÇÃO: calculando a probabilidade de uma X aparecer dado um dos conjuntos (relevante ou irrelevante)

def probDadoconj(palavra, prob_conj, lista_palavras_conj):
    if palavra in lista_palavras_conj:
        return prob_conj[palavra]
    else:
        return 0

#exemplo: print(probDadoconj("la", tabela_relevante, todas_palavras_relevantes))

In [58]:
#SEGUNDA FUNÇÃO: aplicando a suavização de Laplace

def aplicando_laplace(prob_dado_conj, lista_palavras_conj):
    return (prob_dado_conj+1)/(len(lista_palavras_conj)+len(palavras_sem_repeticao))


In [69]:
#FUNÇÃO FINAL: compilado das duas funções anteriores
#essa funçã0 realiza duas vezes as funções escritas anteriormente e depois comparam seus resultados para classificar

def Classificacao(tweet):
    prob_relevante = 1
    lista_tweet = tweet.split()
    for palavra in lista_tweet:
        prob = probDadoconj(palavra, tabela_relevante, todas_palavras_relevantes)
        prob_laplace = aplicando_laplace(prob, todas_palavras_relevantes)
        prob_relevante *= prob_laplace
    probRtweet = prob_relevante*probR
    prob_irrelevante = 1
    for palavra in lista_tweet:
        prob = probDadoconj(palavra, tabela_irrelevante, todas_palavras_irrelevantes)
        prob_laplace = aplicando_laplace(prob, todas_palavras_irrelevantes)
        prob_irrelevante *= prob_laplace
    probIRtweet = prob_irrelevante*probIR
    if probRtweet < probIRtweet: 
        return 0
    else: 
        return 1


In [60]:
train['Classificacao_Naive_Bayes'] = train.Treinamento.apply(Classificacao)
train

,Treinamento,"Classificação (relevante = 1, não relevante = 0)",Classificacao_Naive_Bayes
0,tô vendo la casa de papel mdssss o primeiro ep...,1.0,1
1,agora vou assistir lá casa de papel e dormir p...,0.0,0
2,dedo coçando pra assistir lá casa de papel mas...,0.0,0
3,la casa de papel me faz torcer pros bandidos,1.0,1
4,la casa de papel acabou comigo me encontro des...,1.0,1
...,...,...,...
295,tô aqui assistindo la casa de papel mesmo depo...,0.0,0
296,eu achei que nenhuma morte de la casa de papel...,1.0,1
297,caralho eu to chorando igual uma vagabunda com...,1.0,1
298,cara eu nunca chorei tanto quanto eu chorei ol...,1.0,1


In [61]:
#Porcentagem de acerto de acerto no conjunto de Treino:

tp=train.loc[(train['Classificacao_Naive_Bayes']==1)&(train['Classificação (relevante = 1, não relevante = 0)']==1),:].shape[0]
tn=train.loc[(train['Classificacao_Naive_Bayes']==0)&(train['Classificação (relevante = 1, não relevante = 0)']==0),:].shape[0]
print('A acurácia do modelo foi de: ',100*(tp+tn)/train.shape[0],'%')

A acurácia do modelo foi de:  94.0 %


# Base de Teste

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [62]:
test['Teste'] = test['Teste'].apply(limpeza_e_emoji)
test['Classificacao_Naive_Bayes'] = test['Teste'].apply(Classificacao)
test

,Teste,"Classificação (relevante = 1, não relevante = 0)",Classificacao_Naive_Bayes
0,quero terminar de ver a nova parte de la casa ...,0,0
1,não existe outra série no mundo que mexa comig...,1,1
2,tá foda de desviar de todos os spoilers de la ...,1,1
3,todoroki_jun né tô me sentindo em lá casa de p...,0,1
4,la casa de papel é tão ruim que nem consegui t...,1,0
...,...,...,...
195,em respeito ao meu casal fav de la casa de pap...,0,1
196,assistindo la casa de papel para poder falar m...,0,1
197,sexo não kk veste a roupa aí que a gente vai m...,0,0
198,meu deus o final de la casa de papel…………,1,1


In [63]:
tp=test.loc[(test['Classificacao_Naive_Bayes']==1)&(test['Classificação (relevante = 1, não relevante = 0)']==1),:].shape[0]
tn=test.loc[(test['Classificacao_Naive_Bayes']==0)&(test['Classificação (relevante = 1, não relevante = 0)']==0),:].shape[0]
print('A acurácia do modelo foi de: ',100*(tp+tn)/test.shape[0],'%')

A acurácia do modelo foi de:  68.5 %


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**